# Implementation of Softmax regression

1. Overview
2. Purpose
3. Preparation
4. Implementation
  - Softmax operator
  - Multi-class cross-entropy
  - Gradient descent
3. Training
  - Digits dataset
  - MNIST database

# 1. Overview
Softmax regression is
- a generalized version of **Logistic regression**
- used for **multi-class** classification (*i.e.*, Logistic regression for binary classification; 0/1 output)
- *a.k.a.* **Multinomial Logistic regression**, 
- composed of **linear weights** and **Softmax operator** (nonlinear)

Suppose we have $K$ class classification task setups:
- Inputs: $\boldsymbol{x} \in \mathbb{R}^{D}$
- Labels: $\boldsymbol{t} \in \{1, \cdots, K\}$
- Dataset: $\mathcal{D}=\{(\boldsymbol{x}_i, \boldsymbol{t}_i)\}_{i=1}^{N}$

Here, we define the model parameters and outputs as follows;
- linear weight for class $i$: $\boldsymbol{w}^{(i)} \in \mathbb{R}^D$
- posterior probability for class $i$: $P(y=i|\boldsymbol{x})$
- Model outputs for input data $\boldsymbol{x}$:
$$
\def\bm#1{{\boldsymbol #1}}
P(y=1|\,\bm{x})=
\frac{\exp({\bm{w}^{(1)\top}\bm{x}})}
{\sum_{j=1}^{K}\exp{(\bm{w}^{(j)\top}\bm{x}})}\\
P(y=2|\,\bm{x})=
\frac{\exp({\bm{w}^{(2)\top}\bm{x}})}
{\sum_{j=1}^{K}\exp{(\bm{w}^{(j)\top}\bm{x}})}\\
\vdots\\
P(y=K|\,\bm{x})=
\frac{\exp({\bm{w}^{(K)\top}\bm{x}})}
{\sum_{j=1}^{K}\exp{(\bm{w}^{(j)\top}\bm{x}})}
$$
where $\sum_{i=1}^{K} P(y=i|\bm{x})=1$

# 2. Purpose
Implementing **Softmax regression** from scratch and understanding **how it works**.

This notebook includes implementations of
- Softmax operator
- Multi-class cross-entropy
- Gradient descent

Also, we will use the following dataset;
- Digits dataset
- MNIST database

# 3. Preparation
Importing python libraries

In [ ]:
import os
import sys
import math
import numpy as np

from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

The following libraries are locally located, meaning that you should register the repository directory so that the interpreter can imports the libraries.

In [ ]:
if "google.colab" in sys.modules:
    from google.colab import drive  # Connecting to google drive
    drive.mount("/content/gdrive")
    %cd /content/gdrive/My Drive/sandbox/rc_tutorial/classification_task/
    # NOTE: change it to your own path
    # e.g., %cd /content/gdrive/My Drive/[PATH_TO_NOTEBOOK]
    from utils.magic import colab_script  # save and load cells
    exec(colab_script)
else:
    sys.path.append(".")
    from utils.magic import run_and_save, load_and_run  # save and load cells

from utils.tqdm import trange  # progress bar
from views.softmax_regression import *  # visualization wrapper
from tests.softmax_regression import *  # check function

# 4. Implementation
## 4.1 Softmax operator

First, let's code the activation function `softmax` used in Softmax regression.

Specification of `softmax`
- Arguments:
$$\bm{A}=(\bm{a}_1,\bm{a}_2, \cdots,\bm{a}_N)^\top\in\mathbb{R}^{N\times K} $$
- Return: 
$$\bm{Y}= (\bm{y}_1  \bm{y}_2,\cdots,\bm{y}_N)^\top\in\mathbb{R}^{N\times K}$$ 
where 
$$
\begin{align*}
y_{nk} &= \text{softmax}(\bm{a}_n)_k \\
\text{softmax}(\bm{a})_k 
&=\frac{\exp (a_{k})} {\Sigma_{i=1}^{K}{\exp (a_{i})}}\\
&=\frac{\exp (-a_{\max})\exp (a_{k})}{\exp (-a_{\max})\Sigma_{i=1}^{K}{\exp (a_{i})}} \\
&=\frac{\exp (a_{k}-a_{\max})} {\Sigma_{i=1}^{K}{\exp (a_{i}-a_{\max})}}
\end{align*}
$$
- Note: $a_{\max}$ is the maximum value of $A$ and used for avoiding overflow

<details>
<summary>
Hints
</summary>

- Use `x.shape` to check the dimension.
- Use `np.max` for $a_{\max}$.
    - Parameter `axis` specifies the axis along which to operate.
    - Read [Documentation](https://numpy.org/doc/stable/reference/generated/numpy.maximum.html).
- Use `np.exp` for $\exp$.
- Use `np.sum` or `x.sum(...)` for summation.
    - If `keepdims` is set to `True`, axes that are reduced are left in the result as dimensions with size one.
    - Read [Documentation](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).
- You can also use `x[..., None]` which is equivalent to `keepdims=1`.

In [ ]:
%%run_and_save ./answer/softmax.py

def softmax(a):
    a_max = ... # TODO use a.max and axis option
    a_exp = ... # TODO
    y = ... # TODO
    return y

You will see `OK!` if your code is correct.

In [ ]:
# check softmax
test_softmax(softmax)

## 4.2 Multi-class cross-entropy
Next, let's try to implement a code to calculate cross entropy between model outputs and targets.

Specification of `cross_entropy`
- Arguments
  - Model outputs:
  $$Y=(\bm{y}_1,\bm{y}_2,\cdots,\bm{y}_N)^\top\in \mathbb{R}^{N\times K}$$
  - Teacher labels:
$$
T=(\bm{t}_1,\bm{t}_2,\cdots,\bm{t}_N)^\top\in \mathbb{R}^{N\times K}
$$
- Return
  - Cross entropy: $L \in \mathbb{R}$
  $$L=-\frac{1}{N}\sum_{n=1}^N \sum_i \bm t_{n,i} \log \bm y_{n,i}$$

- Note
  - Clip `Y` to avoid `nan` or `inf`


<details>
<summary>
Hints
</summary>

- Use `np.log` for $\log$.
- Use `np.mean` or `np.average`.
- Use `np.clip` and restrict the range of $\bm{Y}$ value to fit within $[\epsilon, 1 - \epsilon]$ where $\epsilon \ll 1$.


In [ ]:
%%run_and_save ./answer/cross_entropy.py

def cross_entropy(y, t, ep=1e-10):
    y_clip = ... # TODO
    L = ... # TODO
    return L

In [ ]:
# check cross entropy
test_cross_entropy(cross_entropy)

## 4.3 Gradient descent
Then, let's implement a class method named `predict` `gradient` and `step` in `SoftmaxRegression` to update the parameters based on the gradient for cross-entropy loss used in optimization.

Model parameters in `SoftmaxRegression`
- weight (`SoftmaxRegression.W`): $\bm{W}\in\mathbb{R}^{D\times K}$
- bias (`SoftmaxRegression.b`): $\bm{b}\in\mathbb{R}^{K}$

Specification of `SoftmaxRegression.predict`
- Arguments:
  - Teacher inputs：$\bm{X}=(\bm{x}_1,\bm{x}_2, \cdots,\bm{x}_N)\in\mathbb{R}^{N\times D}$

- Return:
  - Model outputs (prior probabilities): $\bm{Y}=(\bm{y}_1,\bm{y}_2, \cdots,\bm{y}_N)\in\mathbb{R}^{N\times K}$
  $$
  \bm{y}_{i} = \text{softmax}(\bm{W}\bm{x}_i+\bm{b})
  $$

Specification of `SoftmaxRegression.gradient`
- Arguments:
  - Teacher inputs：$\bm{X}$
  - Model outputs: $\bm{Y}$
  - Teacher labels: $\bm{T}\in\mathbb{R}^{N\times K}$
- Return:
  - Gradients for cross-entropy loss:
  $$
  \begin{align*}
  \nabla_{\bm{W}}L&=\frac{1}{N}\bm{X}^\top(\bm{Y}-\bm{T})\\
  \nabla_{\bm{b}}L&=\frac{1}{N}(1,1,...,1)(\bm{Y}-\bm{T})
  \end{align*}
  $$
  where $\bm{W},~\bm{b}$ are model parameters (weight and bias)     


Specification of `SoftmaxRegression.step`
- Arguments:
  - Teacher data: $\bm{X}, \bm{T}$
  - Learning rate： $\epsilon \in \mathbb{R}$
- Return: $\bm{Y}$
- Operation:
  - Updating the parameters with gradient descent:
  $$
  \begin{align*}
  \bm{W} &\leftarrow \bm{W}-\epsilon\nabla_{\bm{W}}L\\
  \bm{b} &\leftarrow\bm{b}-\epsilon\nabla_{\bm{b}}L
  \end{align*}
  $$


<details>
<summary>
Hints
</summary>

- Use `self.rnd.uniform` to initialize `self.W` with distribution $\mathcal{U}([-1/\sqrt{D}, 1/\sqrt{D}])$ [(He, K. *et al*., 2015)](https://arxiv.org/abs/1512.03385).
- Use `x.shape` to check the matrix shape.
- Use `np.dot` for matrix multiplication.
  - Avoid using `for` loops as much as possible.
- Use `np.sum` or `np.average`.

In [ ]:
%%run_and_save ./answer/softmax_regression.py

class Module(object):
    def __init__(self, *args, seed=None, dtype=np.float64, **kwargs):
        self.rnd = np.random.RandomState(seed)
        self.dtype = dtype


class SoftmaxRegression(Module):
    def __init__(self, model_dim, class_num, **kwargs):
        super(SoftmaxRegression, self).__init__(**kwargs)
        bound = math.sqrt(1 / model_dim)  # He, K. et al., 2015
        self.W = ... # TODO initialize self.W with self.rnd.uniform
        self.b = np.zeros(class_num)

    def __call__(self, x):
        return self.predict(x)

    def predict(self, x):
        y = ... # TODO
        return y

    def gradient(self, x, y, t):
        delta = ... # TODO
        N = ... # TODO
        gW = ... # TODO
        gb = ... # TODO
        return gW, gb

    def step(self, x, t, ep):
        y = ... # TODO
        gW, gb = self.gradient(x, y, t) 
        self.W = ... # TODO
        self.b = ... # TODO
        return y

In [ ]:
test_gradient_decent(SoftmaxRegression)

# 5. Training
## 5.1 Stochastic gradient descent
Now, it's time to optimize parameters in the Softmax regression model.

The following scripts implement the algorithm of stochastic gradient descent.
Let's find the shuffling process in the algorithm. 

In [ ]:
%%run_and_save ./answer/training.py

def run_epoch(model, X, Y, train=False, batch_size=100, lr=1.0):
    assert X.shape[0] == Y.shape[0]
    data_size = X.shape[0]
    label_y = np.zeros(data_size, dtype=np.int8)
    sum_loss = 0
    if train:
        indices = np.random.permutation(data_size)  # shuffle the indices
    else:
        indices = np.arange(data_size)
        batch_size = data_size  # skip minibatch
    for idx in range(0, data_size, batch_size):
        x = X[indices[idx:idx+batch_size]]
        t = Y[indices[idx:idx+batch_size]]
        if train:
            y = ... # TODO training phase, use model.step
        else:
            y = ... # TODO evaluation phase, use model.predict
        label_y[idx:idx+batch_size] = ... # TODO use np.argmax to get the output label's position
        loss = ... # TODO calculate cross_entropy between y and t
        sum_loss += loss * x.shape[0]
    label_t = np.argmax(Y[indices], axis=1)
    acc = accuracy_score(label_y, label_t)
    loss = sum_loss / data_size
    return acc, loss


def train_model(model, X_train, Y_train, X_eval, Y_eval, n_epoch=10, **kwargs):
    bins = np.linspace(-5, 5, 201)
    record = defaultdict(list)
    record["bins"] = bins
    pbar = trange(n_epoch)
    for epoch in pbar:
        acc_t, loss_t = run_epoch(
            model, X_train, Y_train, train=True, **kwargs)
        acc_e, loss_e = run_epoch(model, X_eval, Y_eval)
        record["train_acc"].append(acc_t)
        record["train_loss"].append(loss_t)
        record["eval_acc"].append(acc_e)
        record["eval_loss"].append(loss_e)
        w_dist, _ = np.histogram(np.log10(np.abs(model.W)), bins)
        record["w_history"].append(w_dist)
        pbar.set_description("{:.4f}/{:.4f}".format(
            acc_e, np.max(record["eval_acc"])))
    return record


## 5.2 [Digits dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits)
A classic handwritten digits dataset (see the link above).


In [ ]:
%matplotlib inline

from sklearn.datasets import load_digits

X, Y = load_digits(return_X_y=True)

class_num = 10
X = X.reshape(X.shape[0], -1)
Y = Y.astype(np.uint8)
Y = np.eye(class_num)[Y].astype(np.int32)
X_train, X_eval, Y_train, Y_eval = train_test_split(
    X, Y, test_size=0.2, random_state=42)
print(X_train.shape, Y_train.shape, X_eval.shape, Y_eval.shape)
fig = show_dataset(X_train, Y_train, class_num, (8, 8))

Let's specify the hyperparameters `n_epoch`, `batch_size`, and `lr`.

In [ ]:
%matplotlib inline

n_epoch = 100
batch_size = 100
lr = 1.0

pixel_size, class_num = X_train.shape[1], Y_train.shape[1]
model = SoftmaxRegression(pixel_size, class_num)

record = train_model(
    model, X_train, Y_train, X_eval, Y_eval,
    n_epoch=n_epoch, batch_size=batch_size, lr=lr)
fig = show_progress(record)

## 5.3 [MNIST database](http://yann.lecun.com/exdb/mnist/)

A database of handwritten digits, available from this page, has a training set of 60,000 examples, and a test set of 10,000 examples

In [ ]:
%matplotlib inline

from torchvision.datasets import MNIST

D_train = MNIST("./data", train=True, download=True,)
D_eval = MNIST("./data", train=False, download=True,)

def convert(D):
    X, Y = D.data.numpy(), D.targets.numpy()
    X = np.array(X / 255.0, dtype=np.float64).reshape(X.shape[0], -1)
    Y = np.array(Y, dtype=np.uint8)
    Y = np.eye(10)[Y]
    return X, Y

X_train, Y_train = convert(D_train)
X_eval, Y_eval = convert(D_eval)
class_num = Y_train.shape[1]
print(X_train.shape, Y_train.shape, X_eval.shape, Y_eval.shape)
fig = show_dataset(X_train, Y_train, class_num, (28, 28))

Please remember this MNIST result as a reference since we will compare it with one in next notebook.

In [ ]:
%matplotlib inline

lr = 1.0
n_epoch = 20
batch_size = 100

pixel_size, class_num = X_train.shape[1], Y_train.shape[1]
model = SoftmaxRegression(pixel_size, class_num)

record = train_model(
    model, X_train, Y_train, X_eval, Y_eval,
    n_epoch=n_epoch, batch_size=batch_size, lr=lr)

fig = show_progress(record)

## Advanced topics

- Check open-source libraries 
  - [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
  - GPU tools
    - [pytorch](https://pytorch.org/)
    - [tensorflow](https://www.tensorflow.org/)
  - *etc*.
- Try your own datasets
- Find good hyperparameter sets by
  - grid search
  - genetic algorithm
  - [AutoML](https://www.automl.org/)
